In [2]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io
import inspect
import requests
import re

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import base64
import email
from email import policy
from email.parser import BytesParser
from email.mime.text import MIMEText

from bs4 import BeautifulSoup
import dateutil.parser as parser

from gptLearning import *

In [4]:
SCOPES = ['https://www.googleapis.com/auth/gmail.send','https://www.googleapis.com/auth/gmail.readonly']
flow = InstalledAppFlow.from_client_secrets_file(
                'credentials-web1.json', SCOPES)
creds = flow.run_local_server(port=9091, access_type='offline', prompt='consent')

with open('token.json', 'w') as token:
    token.write(creds.to_json())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=494569296721-3ihep25jd9oo5a5tnciieelbiblo6j1f.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A9091%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=MfM9RIcRiuOBBBLpG11lu1IwbRvLko&access_type=offline&prompt=consent


In [5]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import json
import base64
from email.header import decode_header

def get_recent_emails(n, userId='me'):
    """
    获取最近的n封邮件

    功能说明：
        此函数用于通过Gmail API获取指定用户的最近n封邮件信息，并将结果以包含多个字典的列表返回，以json格式进行表示。

    参数情况：
        n: 整数，代表需要查询的邮件个数
        userId: 字符串参数，默认情况下取值为'me'，表示查看我的邮件

    返回结果：
        返回一个包含多个字典的列表，并用json格式进行表示，其中一个字典代表一封邮件信息，每个字典中需要包含邮件的发件人、发件时间、邮件主题和邮件内容四个方面信息

    示例：
        get_recent_emails(5, 'me')
    """
    
    # 从token.json加载用户凭据
    creds = Credentials.from_authorized_user_file('token.json', ['https://www.googleapis.com/auth/gmail.readonly'])

    # 建立Gmail服务连接
    service = build('gmail', 'v1', credentials=creds)

    # 获取邮件列表
    results = service.users().messages().list(userId=userId, maxResults=n).execute()
    messages = results.get('messages', [])

    emails = []

    # 循环处理每封邮件
    for message in messages:
        msg = service.users().messages().get(userId=userId, id=message['id']).execute()

        # 获取邮件发件人、发件时间、邮件主题和邮件内容
        email_data = msg['payload']['headers']
        for values in email_data:
            name = values['name']
            if name == 'From':
                from_name = values['value']
            if name == 'Date':
                date = values['value']
            if name == 'Subject':
                subject, encoding = decode_header(values['value'])[0]
                if isinstance(subject, bytes):
                    subject = subject.decode(encoding if encoding else 'utf8')
        content = msg['snippet']

        # 将邮件信息添加到列表中
        emails.append({
            'from': from_name,
            'date': date,
            'subject': subject,
            'content': content
        })

    return json.dumps(emails, indent=4, ensure_ascii=False)


In [6]:
msg = get_recent_emails(n=6)

In [8]:
print(msg)

[
    {
        "from": "Peter from Codeium <peter@codeium.com>",
        "date": "Wed, 16 Aug 2023 05:21:36 +0000 (UTC)",
        "subject": "Why did you stop using Codeium?",
        "content": "Hi Jordan, Thank you for trying Codeium! We&#39;ve noticed that you have not used the service recently, and would greatly appreciate it if you could please fill out this 30 second, 3 question survey to"
    },
    {
        "from": "\"阿新\" <ax909@163.com>",
        "date": "Tue, 15 Aug 2023 22:27:00 +0800 (CST)",
        "subject": "Re:邀请参加会议",
        "content": "好的，明天吃早点。 在 2023-08-15 22:26:35，jrdnnest@gmail.com 写道： &gt;李沉舟先生， &gt; &gt;我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 &gt; &gt;谢谢！ &gt;黄剑"
    },
    {
        "from": "jrdnnest@gmail.com",
        "date": "Tue, 15 Aug 2023 14:26:35 +0000",
        "subject": "Re:邀请参加会议",
        "content": "李沉舟先生， 我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 谢谢！ 黄剑"
    },
    {
        "from": "jrdnnest@gmail.com",
        "date": "Tue, 15 Aug 2023 14:25:33 +00

In [11]:
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-16k',
    messages =[
        {'role': "system","content": "这是我gmail收到的最近6封邮件：%s" % msg},
        {"role": "system", "content": "邮件是通过Gmail API获取的。"},
        {"role": "user", "content": "请问都有哪些人给我发了邮件？内容是什么？"},
    ]
)

In [12]:
print(response.choices[0].message['content'])

最近6封邮件的发送者和内容如下：

1. 发送者：Peter from Codeium <peter@codeium.com>
   内容：Hi Jordan, Thank you for trying Codeium! We've noticed that you have not used the service recently, and would greatly appreciate it if you could please fill out this 30 second, 3 question survey to

2. 发送者："阿新" <ax909@163.com>
   内容：好的，明天吃早点。 在 2023-08-15 22:26:35，jrdnnest@gmail.com 写道： >李沉舟先生， > >我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 > >谢谢！ >黄剑

3. 发送者：jrdnnest@gmail.com
   内容：李沉舟先生， 我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 谢谢！ 黄剑

4. 发送者：jrdnnest@gmail.com
   内容：李沉舟先生， 我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 谢谢！ 黄剑

5. 发送者："阿新" <ax909@163.com>
   内容：好的，明天早上见，一起吃早点。 在 2023-08-15 22:19:56，jrdnnest@gmail.com 写道： >李沉舟先生， > >我是黄剑，我们公司的机器学习工程师。我想邀请您明天早上9点半到我的办公室参加一次会议，我们可以商量下半年的技术开发计划。希望您能抽出时间来参加。 > >谢谢！ > >黄剑

6. 发送者：jrdnnest@gmail.com
   内容：李沉舟先生， 我是黄剑，我们公司的机器学习工程师。我想邀请您明天早上9点半到我的办公室参加一次会议，我们可以商量下半年的技术开发计划。希望您能抽出时间来参加。 谢谢！ 黄剑


In [13]:
functions_list = [get_recent_emails]

In [14]:
chat_with_model(prompt='您好',functions_list=functions_list,model="gpt-3.5-turbo-16k")

模型回答: 你好！有什么我可以帮助你的？
用户问:请帮我查询下gmail邮箱中最近10封邮件发送人及内容
模型回答: 最近10封邮件的发送人及内容如下：

1. 发件人: Peter from Codeium <peter@codeium.com>
   主题: Why did you stop using Codeium?
   内容: Hi Jordan, Thank you for trying Codeium! We've noticed that you have not used the service recently, and would greatly appreciate it if you could please fill out this 30 second, 3 question survey to

2. 发件人: "阿新" <ax909@163.com>
   主题: Re:邀请参加会议
   内容: 好的，明天吃早点。 在 2023-08-15 22:26:35，jrdnnest@gmail.com 写道： &gt;李沉舟先生， &gt; &gt;我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 &gt; &gt;谢谢！ &gt;黄剑

3. 发件人: jrdnnest@gmail.com
   主题: Re:邀请参加会议
   内容: 李沉舟先生， 我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 谢谢！ 黄剑

4. 发件人: jrdnnest@gmail.com
   主题: Re:邀请参加会议
   内容: 李沉舟先生， 我代表黄剑邀请您于明天早上9点半来我办公室开会，商量下半年技术开发计划。 谢谢！ 黄剑

5. 发件人: "阿新" <ax909@163.com>
   主题: Re:会议邀请
   内容: 好的，明天早上见，一起吃早点。 在 2023-08-15 22:19:56，jrdnnest@gmail.com 写道： &gt;李沉舟先生， &gt; &gt;我是黄剑，我们公司的机器学习工程师。我想邀请您明天早上9点半到我的办公室参加一次会议，我们可以商量下半年的技术开发计划。希望您能抽出时间来参加。 &gt; &gt;谢谢！ &gt; &gt;黄剑

6. 发件人:

In [15]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k",
  messages=[{"role": "system", "content": "你是一个python代码编辑器，你的功能是输出python代码，请勿输出任何和python代码无关的内容"},
            {"role": "user", "content": "请帮我定义一个python函数，输出Hello world字符串，请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档。"}
  ]
)

In [23]:
s = response.choices[0].message['content']
print(s)

下面是一个定义函数并输出"Hello world"字符串的示例代码：

```python
def print_hello_world():
    """
    这是一个用于输出"Hello world"字符串的函数。

    参数：
        无

    返回值：
        无

    示例：
        >>> print_hello_world()
        Hello world
    """
    print("Hello world")
```

你可以使用`print_hello_world()`调用该函数来输出"Hello world"字符串。注意在文档字符串中，我们提供了函数的详细说明，包括函数的作用、参数、返回值和示例使用方法。这样可以帮助其他人更好地理解和使用这个函数。


In [18]:
def extract_code(s):
    """
    如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
    否则，返回原字符串。

    参数:
    s: 输入的字符串。

    返回:
    提取出的代码部分，或原字符串。
    """
    # 判断字符串是否是Markdown格式
    if '```python' in s or 'Python' in s or'PYTHON' in s:
        # 找到代码块的开始和结束位置
        code_start = s.find('def')
        code_end = s.find('```\n', code_start)
        # 提取代码部分
        code = s[code_start:code_end]
    else:
        # 如果字符串不是Markdown格式，返回原字符串
        code = s

    return code

In [24]:
code_s = extract_code(s)

In [25]:
code_s

'def print_hello_world():\n    """\n    这是一个用于输出"Hello world"字符串的函数。\n\n    参数：\n        无\n\n    返回值：\n        无\n\n    示例：\n        >>> print_hello_world()\n        Hello world\n    """\n    print("Hello world")\n'

In [27]:
with open('helloworld.py','w',encoding='utf-8') as f:
    f.write(code_s)

In [28]:
with open('helloworld.py',encoding='utf-8') as f:
    content = f.read()

In [30]:
content

'def print_hello_world():\n    """\n    这是一个用于输出"Hello world"字符串的函数。\n\n    参数：\n        无\n\n    返回值：\n        无\n\n    示例：\n        >>> print_hello_world()\n        Hello world\n    """\n    print("Hello world")\n'

In [32]:
from helloworld import *

In [33]:
print_hello_world()

Hello world


In [3]:
def extract_function_code(s, detail=0,tested=False):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息
    """
    def extract_code(s):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s or 'Python' in s or'PYTHON' in s:
            # 找到代码块的开始和结束位置
            code_start = s.find('def')
            code_end = s.find('```\n', code_start)
            # 提取代码部分
            code = s[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s

        return code
    
    # 提取代码字符串
    code = extract_code(s)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 将函数写入本地
    if tested==False:
        with open('./functions/untested functions/%s_module.py' % function_name, 'w', encoding='utf-8') as f:
            f.write(code)
    else:
        with open('./functions/tested functions/%s_module.py' % function_name, 'w', encoding='utf-8') as f:
            f.write(code)
    
    
    # 执行该函数
    try:
        exec(code, globals())
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1 and tested==False:
        with open('./functions/untested functions/%s_module.py' % function_name, encoding='utf-8') as f:
            content = f.read()
        print(content)
    elif detail == 1 and tested==True:
        with open('./functions/tested functions/%s_module.py' % function_name, encoding='utf-8') as f:    
            content = f.read()
        print(content)

In [46]:
extract_function_code(s)

The function name is:print_hello_world


In [40]:
print_hello_world?

Signature: print_hello_world()
Docstring:
这是一个用于输出"Hello world"字符串的函数。

参数：
    无

返回值：
    无

示例：
    >>> print_hello_world()
    Hello world
File:      Dynamically generated function. No source code available.
Type:      function

In [41]:
import sys

In [47]:
sys.path.insert(0,'./functions/untested functions/')

In [52]:
from print_hello_world_module import *

In [53]:
print_hello_world?

Signature: print_hello_world()
Docstring:
这是一个用于输出"Hello world"字符串的函数。

参数：
    无

返回值：
    无

示例：
    >>> print_hello_world()
    Hello world
File:      ~/gpt/9t/functions/untested functions/print_hello_world_module.py
Type:      function

In [50]:
del print_hello_world

In [55]:
import print_hello_world_module as phwm

In [57]:
phwm.print_hello_world()

Hello world


In [58]:
def get_latest_email(userId):
    """
    查询Gmail邮箱中最后一封邮件信息
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    注意，当查询我的邮箱时，userId需要输入'me'；
    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)
    
    # 列出用户的一封最新邮件
    results = service.users().messages().list(userId=userId, maxResults=1).execute()
    messages = results.get('messages', [])

    # 遍历邮件
    for message in messages:
        # 获取邮件的详细信息
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        
    return json.dumps(msg)

In [60]:
code = inspect.getsource(get_latest_email)

In [61]:
type(code)

str

In [63]:
with open('./functions/tested functions/%s_module.py' % 'get_latest_email','w',encoding='utf-8') as f:
    f.write(code)

In [68]:
with open('./functions/tested functions/get_latest_email_module.py', 'r',encoding='utf-8') as f:
    content = f.read()

In [69]:
content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [16]:
system_content = "我已经取得gmail api授权，授权文件为本地文件token.json"

In [17]:
user_example_content = "请帮我编写一个python函数，用于查看gmail邮箱中最后一封邮件内容，函数要求如下：\
                        1.函数参数为userId，必要参数，userId为字符串参数，默认情况下取值为'me‘，表示查看我自己的邮箱；\
                        2.函数返回结果为包含最后一封邮件信息的对象，返回结果必须是json格式对象；\
                        3.函数返回结果中必须包含邮件的发件人、收件人、主题、正文、附件等信息；\
                        4.请将全部功能封装在一个函数中；\
                        5.请在函数编写过程中，在函数内部加入中文编写的详细的说明文档，用于说明函数功能、函数参数、函数返回结果等信息。"

In [18]:
with open("./functions/tested functions/%s_module.py" % "get_latest_email","r",encoding="utf-8") as f:
    assistant_example_content = f.read()

In [19]:
assistant_example_content

'def get_latest_email(userId):\n    """\n    查询Gmail邮箱中最后一封邮件信息\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含最后一封邮件全部信息的对象，该对象由Gmail API创建得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 列出用户的一封最新邮件\n    results = service.users().messages().list(userId=userId, maxResults=1).execute()\n    messages = results.get(\'messages\', [])\n\n    # 遍历邮件\n    for message in messages:\n        # 获取邮件的详细信息\n        msg = service.users().messages().get(userId=\'me\', id=message[\'id\']).execute()\n        \n    return json.dumps(msg)\n'

In [20]:
user_content = "请帮我编写一个python函数，用于查看gmail邮箱中邮件数量，函数要求如下：\
                1.函数参数为userId，必要参数，userId为字符串参数，默认情况下取值为'me‘，表示查看我自己的邮箱；\
                2.函数返回结果为包含最后一封邮件信息的对象，返回结果必须是json格式对象；\
                3.请将全部功能封装在一个函数中；\
                4.请在函数编写过程中，在函数内部加入中文编写的详细的说明文档，用于说明函数功能、函数参数、函数返回结果等信息。"

In [21]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-16k",
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "name": "example_user", "content":user_example_content},
        {"role": "assistant", "name": "eaxmple_assistant", "content": assistant_example_content},
        {"role": "user",  "content":user_content}
    ]
)

In [22]:
s = response.choices[0].message.content

In [23]:
s

'def count_emails(userId):\n    """\n    获取Gmail邮箱中的邮件数量\n    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\\\n    注意，当查询我的邮箱时，userId需要输入\'me\'；\n    :return：包含邮件数量的对象，该对象由Gmail API返回得到，且保存为JSON格式\n    """\n    # 从本地文件中加载凭据\n    creds = Credentials.from_authorized_user_file(\'token.json\')\n    \n    # 创建 Gmail API 客户端\n    service = build(\'gmail\', \'v1\', credentials=creds)\n    \n    # 获取邮件数量\n    response = service.users().messages().list(userId=userId).execute()\n    count = response.get(\'resultSizeEstimate\', 0)\n    \n    return json.dumps({"count": count})'

In [24]:
extract_function_code(s,detail=0)

The function name is:count_emails


In [25]:
extract_function_code(s,detail=1)

def count_emails(userId):
    """
    获取Gmail邮箱中的邮件数量
    :param userId: 必要参数，字符串类型，用于表示需要查询的邮箱ID，\
    注意，当查询我的邮箱时，userId需要输入'me'；
    :return：包含邮件数量的对象，该对象由Gmail API返回得到，且保存为JSON格式
    """
    # 从本地文件中加载凭据
    creds = Credentials.from_authorized_user_file('token.json')
    
    # 创建 Gmail API 客户端
    service = build('gmail', 'v1', credentials=creds)
    
    # 获取邮件数量
    response = service.users().messages().list(userId=userId).execute()
    count = response.get('resultSizeEstimate', 0)
    
    return json.dumps({"count": count})


In [26]:
get_email_count(userId='me')

'{"email_count": 201}'

In [103]:
functions_list = [get_email_count]

In [104]:
functions = auto_functions(functions_list=functions_list)

In [105]:
functions

[{'name': 'get_email_count',
  'description': '查询Gmail邮箱中邮件数量',
  'parameters': {'type': 'object',
   'properties': {'userId': {'type': 'string',
     'description': "用于表示需要查询的邮箱ID，注意，当查询我的邮箱时，userId需要输入'me'"}},
   'required': ['userId']}}]

In [106]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": '请帮我查下邮箱里现在总共有多少封邮件'}],
        functions=functions,
        function_call="auto",  
    )

In [107]:
response

<OpenAIObject chat.completion id=chatcmpl-7oCGCDgzGOUuR6cydXNrtrJHUddTv at 0x1522d7fb0> JSON: {
  "id": "chatcmpl-7oCGCDgzGOUuR6cydXNrtrJHUddTv",
  "object": "chat.completion",
  "created": 1692197884,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_email_count",
          "arguments": "{\n  \"userId\": \"me\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 93,
    "completion_tokens": 16,
    "total_tokens": 109
  }
}

In [108]:
chat_with_model?

Signature: chat_with_model(prompt, functions_list=None, model='gpt-3.5-turbo-16k')
Docstring: <no docstring>
File:      ~/gpt/9t/gptLearning.py
Type:      function

In [109]:
chat_with_model(prompt='你好',functions_list = functions_list)

模型回答: 你好！有什么我可以帮助你的吗？
用户问:请帮我查询下我的邮箱里有多少邮件？
模型回答: 您的邮箱中共有29封邮件。
用户问:退出
